In [1]:
import pandas as pd
import scipy.stats as st
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as dtc
from sklearn.metrics import accuracy_score, mean_squared_error
from joblib import dump, load
from sklearn import tree
import math
import random
%matplotlib inline

In [2]:
# load data
df = pd.read_csv('health care diabetes.csv')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
# global variables
X = df.drop(columns=['Outcome']) # all input values
y = df['Outcome']

clusters = [] # stores x values of data
m_init = 1
b_init = 0
learning_rate = 0.01 # default
points = []

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
print(f"X train: {X_train}\n")
print(f"X test: {X_test}\n")
print(f"y train: {y_train}\n")
print(f"y test: {y_test}\n")

model = LogisticRegression()
model.fit(X_train, y_train)
print(f"Accuracy: {model.score(X_test, y_test)}")

X train:      Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
357           13      129              0             30        0  39.9   
551            3       84             68             30      106  31.9   
365            5       99             54             28       83  34.0   
685            2      129             74             26      205  33.2   
670            6      165             68             26      168  33.6   
..           ...      ...            ...            ...      ...   ...   
492            4       99             68             38        0  32.8   
226            0      101             76              0        0  35.7   
697            0       99              0              0        0  25.0   
683            4      125             80              0        0  32.3   
315            2      112             68             22       94  34.1   

     DiabetesPedigreeFunction  Age  
357                     0.569   44  
551                     0.59

C:\Users\ezrab\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
model.predict(X_test)

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0], dtype=int64)

In [6]:
def sigmoid_function(x, m, b):
    return 1/(1 + math.exp(-1*(m*x+b)))
              
# optimize model using stochastic gradient descent
        

def cluster_points():
    # increment by some x and use a circle with radius x/2 to capture points
    print("Clustering points")
    increment = 10 # diameter of circle
    start = min(X_adjusted)
    end = max(X_adjusted)
    # (x - h)^2 + (y - k)^2 <= r^2
    radius = increment/2
    X_center = increment/2
    y_center = sigmoid_function(X_center, m_init, b_init)
    
    for i in range(start, end, increment):
        temp = []
        for j in range(start, end):
            if ((X_adjusted[j] - X_center)**2 + (y_adjusted[j] - y_center)**2) <= (radius)**2:
                temp.append(X_adjusted[j])
            
        if len(temp) > 0:
            clusters.append(temp)
        
        X_center = X_center + increment
        y_center = sigmoid_function(X_center, m_init, b_init)
        print(f"{X_center}, {y_center}")
        
#     remove_duplicates()
    print(clusters)
    
def remove_duplicates():
    temp = dict()
    for i in range(0, len(clusters)):
        for j in range(0, len(clusters[i])):
            if clusters[i][j] not in temp.keys():
                temp[clusters[i][j]] = 1
            else:
                clusters[i].pop(j)
        
    
def random_cluster_points(): # empty range for randrange() (0,0,0)
    print("Selecting a random point per cluster")
    for cluster in clusters:
        if len(cluster) >= 1:
            if len(cluster) > 1:
                i = random.randint(0, len(cluster)-1)
                points.append(cluster[i])
            else:
                points.append(cluster[0])

    print(points)
    

# partial derivative of loss function with respect to slope
def derivative_m(x, y, m, b):
    return (-2*x)*(y-(b+m*x))
    
    
# partial derivative of loss function with respect to y-intercept
def derivative_b(x, y, m, b):
    return -2*(y-(b+m*x))

def stochastic_gradient_descent(m, b):
    print("Performing SGD on selected cluster points")
    cluster_points()
    random_cluster_points()
    for point in points:
        
        global m_init
        global b_init
        
        y = sigmoid_function(point, m_init, b_init)
        
        adjusted_m = derivative_m(point, y, m_init, b_init)
        adjusted_b = derivative_b(point, y, m_init, b_init)
        
        print(adjusted_m)
        print(adjusted_b)
        
        
        m_init = m_init - adjusted_m * learning_rate
        b_init = b_init - adjusted_b * learning_rate
        print(f"New slope: {m_init}")
        print(f"New intercept: {b_init}")
        
def calibration_process():
    stochastic_gradient_descent(m_init, b_init)
    print(f"Accuracy: {model.score(X_test, y_test)}")

In [11]:
y_test

173    0
339    1
216    1
650    0
485    1
      ..
487    0
7      0
368    0
447    0
655    1
Name: Outcome, Length: 192, dtype: int64

In [21]:
# Tasks for now:
# single b, weights
# refresh logistic regression ^
# logistic regression sklearn, compare my rms to built in ^
# ensure model has converged
# calculate root mean square error for each point ^
# convergence - rms error converges to smallest value it could achieve ^
# build logistic regression consider all 8 columns ^
# skip clustering

def rms_error():
    err = 0
    old_err = 0
    i = 0
    for prediction in model.predict(X_test):
        for val in y_test:
            new_err = (prediction - val)**2
            err = err + new_err
            i = i+1
    
    mean_squared_err = math.sqrt(err/y_test.size)
    
    print(f"N = {y_test.size}")
    print(f"∑(x-xi)² = {err}")
    print(f"Mean Squared Error = {mean_squared_err}")

    return mean_squared_err
    
print(f"Sklearn Mean squared error: {mean_squared_error(model.predict(X_test), y_test)}")
print(f"My Mean squared error formula: {rms_error()}")

Sklearn Mean squared error: 0.21875
N = 192
∑(x-xi)² = 15552
Mean Squared Error = 9.0
My Mean squared error formula: 9.0
